In [3]:
%pip install haystack-ai
%pip install pymed

  Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
  Using cached h11-0.14.0-py3-none-any.whl (58 kB)
   ---------------------------------------- 0.0/239.6 kB ? eta -:--:--
   ---------------------------------------- 239.6/239.6 kB 7.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/226.7 kB ? eta -:--:--
   --------------------------------------- 226.7/226.7 kB 13.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/57.0 kB ? eta -:--:--
   ---------------------------------------- 57.0/57.0 kB ? eta 0:00:00
   ---------------------------------------- 0.0/41.1 kB ? eta -:--:--
   ---------------------------------------- 41.1/41.1 kB ? eta 0:00:00
   ---------------------------------------- 0.0/75.9 kB ? eta -:--:--
   ---------------------------------------- 75.9/75.9 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/76.9 kB ? eta -:--:--
   ---------------------------------------- 76.9/76.9 kB 4.2 MB/s eta 0:00:00
Note: you may n

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

huggingfacehub_api_token = os.getenv("HUGGINGFACEHUB_API_TOKEN")

In [5]:
from pymed import PubMed
from typing import List
from haystack import component
from haystack import Document

C:\Users\srija\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
pubmed = PubMed(tool="Haystack2.0Prototype", email="srijanshovit@gmail.com")


In [7]:
def documentize(article):
  return Document(content=article.abstract, meta={'title': article.title, 'keywords': article.keywords})

@component
class PubMedFetcher():

  @component.output_types(articles=List[Document])
  def run(self, queries: list[str]):
    cleaned_queries = queries[0].strip().split('\n')

    articles = []
    try:
      for query in cleaned_queries:
        response = pubmed.query(query, max_results = 1)
        documents = [documentize(article) for article in response]
        articles.extend(documents)
    except Exception as e:
        print(e)
        print(f"Couldn't fetch articles for queries: {queries}" )
    results = {'articles': articles}
    return results

In [14]:
from haystack.components.generators import HuggingFaceTGIGenerator
from haystack.utils import Secret


keyword_llm = HuggingFaceTGIGenerator("mistralai/Mistral-7B-Instruct-v0.2")
keyword_llm.warm_up()

llm = HuggingFaceTGIGenerator("mistralai/Mistral-7B-Instruct-v0.2")
llm.warm_up()

tokenizer_config.json: 100%|██████████| 1.46k/1.46k [00:00<?, ?B/s]
C:\Users\srija\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\srija\.cache\huggingface\hub\models--mistralai--Mistral-7B-Instruct-v0.2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/e

In [15]:
from haystack import Pipeline
from haystack.components.builders.prompt_builder import PromptBuilder

keyword_prompt_template = """
Your task is to convert the following question into 3 keywords that can be used to find relevant medical research papers on PubMed.
Here is an examples:
question: "What are the latest treatments for major depressive disorder?"
keywords:
Antidepressive Agents
Depressive Disorder, Major
Treatment-Resistant depression
---
question: {{ question }}
keywords:
"""

prompt_template = """
Answer the question truthfully based on the given documents.
If the documents don't contain an answer, use your existing knowledge base.

q: {{ question }}
Articles:
{% for article in articles %}
  {{article.content}}
  keywords: {{article.meta['keywords']}}
  title: {{article.meta['title']}}
{% endfor %}

"""
keyword_prompt_builder = PromptBuilder(template=keyword_prompt_template)

prompt_builder = PromptBuilder(template=prompt_template)
fetcher = PubMedFetcher()

pipe = Pipeline()

pipe.add_component("keyword_prompt_builder", keyword_prompt_builder)
pipe.add_component("keyword_llm", keyword_llm)
pipe.add_component("pubmed_fetcher", fetcher)
pipe.add_component("prompt_builder", prompt_builder)
pipe.add_component("llm", llm)

pipe.connect("keyword_prompt_builder.prompt", "keyword_llm.prompt")
pipe.connect("keyword_llm.replies", "pubmed_fetcher.queries")

pipe.connect("pubmed_fetcher.articles", "prompt_builder.articles")
pipe.connect("prompt_builder.prompt", "llm.prompt")


In [16]:
from IPython.display import display, HTML

def ask(question):
  output = pipe.run(data={"keyword_prompt_builder":{"question":question},
                          "prompt_builder":{"question": question},
                          "llm":{"generation_kwargs": {"max_new_tokens": 500}}})
  print(question)
  print(output['llm']['replies'][0])
  # display(HTML(f'<div style="color: blue">{output["llm"]['replies'][0]}</div>'))


In [17]:
ask("How are mRNA vaccines being used for cancer treatment?")

How are mRNA vaccines being used for cancer treatment?
a: mRNA vaccines are being used for cancer treatment by delivering mRNA encoding tumor antigens to stimulate an immune response against cancer cells. The documents do not provide specific information on how mRNA vaccines are being used for cancer treatment, but they do mention the use of mRNA for therapeutic interventions and the development of polymeric architectures for mRNA delivery. The documents do not mention cancer treatment specifically in relation to these polymeric architectures, but they do suggest that they can be tailored for mRNA protection, loading efficacy, and targeted release, which could be useful for cancer treatment applications. Overall, while the documents do not directly answer the question, they do provide some context and background information on the use of mRNA for therapeutic applications, including cancer treatment.


In [30]:
%pip install mermaid
%pip install pygraphviz


     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
     -- ------------------------------------- 0.1/1.2 MB 544.7 kB/s eta 0:00:03
     ---------------------- ----------------- 0.7/1.2 MB 4.6 MB/s eta 0:00:01
     ---------------------------------------  1.2/1.2 MB 7.5 MB/s eta 0:00:01
     ---------------------------------------- 1.2/1.2 MB 5.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/840.9 kB ? eta -:--:--
     ------------------------------------  839.7/840.9 kB 51.8 MB/s eta 0:00:01
     ------------------------------------- 840.9/840.9 kB 17.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dep

DEPRECATION: mermaid 0.3.2 has a non-standard dependency specifier torch>=1.7torchvision. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of mermaid or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063



  Using cached pygraphviz-1.12.tar.gz (104 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build pygraphviz


  error: subprocess-exited-with-error
  
  × Building wheel for pygraphviz (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [49 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-311
      creating build\lib.win-amd64-cpython-311\pygraphviz
      copying pygraphviz\agraph.py -> build\lib.win-amd64-cpython-311\pygraphviz
      copying pygraphviz\graphviz.py -> build\lib.win-amd64-cpython-311\pygraphviz
      copying pygraphviz\scraper.py -> build\lib.win-amd64-cpython-311\pygraphviz
      copying pygraphviz\testing.py -> build\lib.win-amd64-cpython-311\pygraphviz
      copying pygraphviz\__init__.py -> build\lib.win-amd64-cpython-311\pygraphviz
      creating build\lib.win-amd64-cpython-311\pygraphviz\tests
      copying pygraphviz\tests\test_attribute_defaults.py -> build\lib.win-amd64-cpython-311\pygraphviz\tests
      copying pygraphviz\tests\test_clear.py -> build\lib.

In [34]:
pipe.draw("img.jpg")